# Plot augmented Nek temperature with PyVista

Set `MESH_FILE` to the first augmented file (the one that includes the mesh).
If you want to plot a later timestep, set `FIELD_FILE` to that file and keep `MESH_FILE`
pointing to the first file.


In [1]:
from pathlib import Path
import numpy as np
import pyvista as pv
from mpi4py import MPI

from pysemtools.datatypes.msh import Mesh
from pysemtools.datatypes.field import FieldRegistry
from pysemtools.io.ppymech.neksuite import pynekread


In [2]:
PROJECT_ROOT = Path.cwd()
MESH_FILE = PROJECT_ROOT / "data/processed_nek/phi0.40/h400x025_ref/premix_aug0.f00249"
FIELD_FILE = MESH_FILE  # change to another timestep if needed

if not MESH_FILE.exists():
    raise FileNotFoundError(MESH_FILE)
if not FIELD_FILE.exists():
    raise FileNotFoundError(FIELD_FILE)


In [3]:
comm = MPI.COMM_SELF
msh = Mesh(comm, create_connectivity=False)
fld = FieldRegistry(comm)

pynekread(str(MESH_FILE), comm, msh=msh)
pynekread(str(FIELD_FILE), comm, msh=msh, fld=fld, overwrite_fld=True)


2026-01-27 10:50:06,156  pynekread             INFO      Reading file: /media/alexandros/OS/Users/alexp/Documents/Bachelor Thesis/Code/data/processed_nek/phi0.40/h400x025_ref/premix_aug0.f00249
2026-01-27 10:50:06,174  Mesh                  INFO      Mesh object initialized from coordinates with type: float64 - Elapsed time: 0.0074678209999999995s
2026-01-27 10:50:06,174  pynekread             INFO      File successfully read - Elapsed time: 0.01865022s
2026-01-27 10:50:06,175  pynekread             INFO      Reading file: /media/alexandros/OS/Users/alexp/Documents/Bachelor Thesis/Code/data/processed_nek/phi0.40/h400x025_ref/premix_aug0.f00249
2026-01-27 10:50:06,186  Mesh                  INFO      Mesh object initialized from coordinates with type: float64 - Elapsed time: 0.00786448s
2026-01-27 10:50:06,186  pynekread             INFO      Overwriting fld object
2026-01-27 10:50:06,211  Field                 INFO      Field registry updated with: ['u', 'v', 'p', 't', 's0', 's1', 's2'

In [4]:
def build_pyvista_grid(msh):
    x = np.asarray(msh.x)
    y = np.asarray(msh.y)
    z = np.asarray(msh.z)

    nelv, nz, ny, nx = x.shape

    points = np.column_stack([
        x.ravel(order="C"),
        y.ravel(order="C"),
        z.ravel(order="C"),
    ])

    def idx(e, k, j, i):
        return (((e * nz + k) * ny + j) * nx + i)

    cells = []
    cell_types = []

    if nz == 1:
        for e in range(nelv):
            for j in range(ny - 1):
                for i in range(nx - 1):
                    n0 = idx(e, 0, j, i)
                    n1 = idx(e, 0, j, i + 1)
                    n2 = idx(e, 0, j + 1, i + 1)
                    n3 = idx(e, 0, j + 1, i)
                    cells.extend([4, n0, n1, n2, n3])
                    cell_types.append(pv.CellType.QUAD)
    else:
        for e in range(nelv):
            for k in range(nz - 1):
                for j in range(ny - 1):
                    for i in range(nx - 1):
                        n0 = idx(e, k, j, i)
                        n1 = idx(e, k, j, i + 1)
                        n2 = idx(e, k, j + 1, i + 1)
                        n3 = idx(e, k, j + 1, i)
                        n4 = idx(e, k + 1, j, i)
                        n5 = idx(e, k + 1, j, i + 1)
                        n6 = idx(e, k + 1, j + 1, i + 1)
                        n7 = idx(e, k + 1, j + 1, i)
                        cells.extend([8, n0, n1, n2, n3, n4, n5, n6, n7])
                        cell_types.append(pv.CellType.HEXAHEDRON)

    if not cells:
        raise RuntimeError("UnstructuredGrid has no cells – check mesh dimensions.")

    grid = pv.UnstructuredGrid(
        np.asarray(cells, dtype=np.int64),
        np.asarray(cell_types, dtype=np.uint8),
        points,
    )
    return grid


In [5]:
grid = build_pyvista_grid(msh)
T = np.asarray(fld.fields["temp"][0])
grid.point_data["T"] = T.ravel(order="C")


In [6]:
# Compute temperature gradient (dTdx)
grid_g = grid.compute_derivative(scalars="T", gradient=True)
grad = grid_g.point_data["gradient"]
grid_g.point_data["dTdx"] = grad[:, 0]


In [7]:
pv.set_plot_theme("document")
plotter = pv.Plotter()
plotter.add_mesh(grid_g, scalars="dTdx", cmap="inferno")
plotter.add_axes()
plotter.show()


Widget(value='<iframe src="http://localhost:36929/index.html?ui=P_0x7a51a8effb90_0&reconnect=auto" class="pyvi…

In [8]:
z = np.asarray(msh.z)
print(np.min(z), np.max(z))
x = np.asarray(msh.x)
print(np.min(x), np.max(x))
y = np.asarray(msh.y)
print(np.min(y), np.max(y))

0.0 0.0
300.0 400.0
0.0 25.0
